In [1]:
pip install geopy scipy



[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ------------ -------------------------- 41.0/125.4 kB 487.6 kB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB 1.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from geopy.distance import geodesic

def geographic_median(latitudes, longitudes):
    """
    Calculate the geographic median using the centroid approximation.
    """
    if not latitudes or not longitudes:
        return None, None  # Return None if no valid points

    # Calculate centroid as a simple average of latitudes and longitudes
    median_lat = sum(latitudes) / len(latitudes)
    median_lon = sum(longitudes) / len(longitudes)
    return median_lat, median_lon

# Load the dataset
file_path = r'C:\Users\ajmal\OneDrive\Documents\NCOL\ICS Tracker\Raw_Pincode_Downloaded.csv'
data = pd.read_csv(file_path, low_memory=False)

# Validate and clean Latitude/Longitude
data['Latitude'] = pd.to_numeric(data['Latitude'], errors='coerce')
data['Longitude'] = pd.to_numeric(data['Longitude'], errors='coerce')

# Remove rows with NA in Latitude or Longitude
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Remove rows with invalid Latitude and Longitude ranges
data = data[(data['Latitude'] >= 6.0) & (data['Latitude'] <= 37.1) & 
            (data['Longitude'] >= 68.1) & (data['Longitude'] <= 97.4)]

# Group by Pincode, District, and StateName
results = []
last_median_lat, last_median_lon = None, None  # Variables to track the last median latitude and longitude
for (pincode, district, state), group in data.groupby(['Pincode', 'District', 'StateName']):
    latitudes = group['Latitude'].tolist()
    longitudes = group['Longitude'].tolist()
    try:
        median_lat, median_lon = geographic_median(latitudes, longitudes)
        last_median_lat, last_median_lon = median_lat, median_lon  # Update the last median values
        results.append({
            'Pincode': pincode,
            'District': district,
            'State': state,
            'Latitude': median_lat,
            'Longitude': median_lon
        })
    except Exception as e:
        print(f"Error processing Pincode={pincode}, District={district}, State={state}: {e}")
        print(f"Last Latitudes: {latitudes}, Last Longitudes: {longitudes}")

# Print the last median latitude and longitude
print(f"Last Median Latitude: {last_median_lat}, Last Median Longitude: {last_median_lon}")

# Convert results to a DataFrame
median_data = pd.DataFrame(results)

# Save to a CSV file
output_file_path = r'C:\Users\ajmal\Downloads\pincode_geographic_median_summary.csv'
median_data.to_csv(output_file_path, index=False)

print(f"Output file saved as {output_file_path}")


Last Median Latitude: 26.297472666666668, Last Median Longitude: 86.52814066666667
Output file saved as pincode_geographic_median_summary.csv


In [18]:
output_file_path = r'C:\Users\ajmal\Downloads\pincode_geographic_median_summary.csv'
median_data.to_csv(output_file_path, index=False)